In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import io


In [11]:
#Read the csv file with utf-8 decoding
df = pd.read_csv('C:/Users/kumarr/Desktop/Personal/MScDataAnalytics_CCT/Capstone/Colonoscopyrecords.csv')
df.head()

,ID,Age,proceduredate,Gender,CLINICID
0,1,69,19/03/2013,M,19
1,2,69,12/03/2013,F,19
2,3,69,05/03/2013,F,19
3,4,69,12/03/2013,M,19
4,5,69,12/03/2013,M,19
